# Flyhweel Software Developer Kit (SDK) Basics and Helpful Hints

**Date modified:** 28-July-2022 <br>
**Authors:** Amy Hegarty, Lena Sherbakov, Intermountain Neuroimaging Consortium

**Description:** <br>
The following includes a set of basic SDK commands for Flywheel. Examples included below:

 - Basic Flywheel Heirarchy (finding a session)

**Links** <br>

 - Flywheel SDK documentation [here](https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/python/index.html)
 - Flywheel python SDK examples [here](https://gitlab.com/flywheel-io/public/flywheel-tutorials/-/blob/master/python/)

In [3]:
print("Welcome to Intermountain Neuroimaging Consortium!")

Welcome to Intermountain Neuroimaging Consortium!


## Import Python Dependencies

In [ ]:
import os
import logging
import flywheel

## Define Supporting Functions

## Flywheel API Key and Client
An API Key is required to interact with the datasets on flywheel db. More on this in the Flywheel SDK doc [here](https://flywheel-io.gitlab.io/product/backend/sdk/branches/master/python/getting_started.html#api-key). Here we pull the API key directly from the user's os enviornment, set within the flywheel command line interface.  

In [ ]:
# Instantiate a logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
log = logging.getLogger('root')

In [ ]:
# Create client, using CLI credentials
fw = flywheel.Client()

# who am I logged in as?
log.info('You are now logged in as %s to %s', fw.get_current_user()['email'], fw.get_config()['site']['api_url'])

## Viewing a Flywheel Project
The `fw.lookup` function can be used to find any container in flywheel by path. This is an easy way to find projects, subjects, or sessions. Lets view a Flywheel project and list all sessions and their scan date.

In [ ]:
project = fw.lookup('<group>/<project>')

# lookup function pulls a lite version of the project object. Get the actual project object...
project = fw.get(project.id)

# loop through all project sessions
for session in project.sessions.find():

    # Loop through all sessions in the project container.
    dt = session.timestamp
    if not dt:
        dt = "NAN"
    else:
        dt = dt.strftime(" %x %X")
    print('%s: Subject: %s Session: %s\tScanning Date: %s' % (session.id, session.subject.label, session.label, dt))


## Viewing a Flywheel Analysis 
Similarly, we can view analyses that have been either uploaded to Flywheel or generated by running flywheel gears. Here, lets look for all analyses attached to a single session.

In [ ]:
# start by finding a session of interest
session = fw.lookup('<group>/<project>/<subject>/<session>')

# we need to take an extra step to get the full session object, not just basic info 
session_object = fw.get_session(session.id)

# print all the analyses for this session
for analysis in session_object.analyses:
    if hasattr(analysis.job,'state'):
        print('%s: %s %s' % (analysis.id, analysis.label, analysis.job.state))
    else:
        print('%s: %s' % (analysis.id, analysis.label))

## Update Metadata In Flywheel
In this example, lets rename an acquisition to add an "include-BIDS" suffix used to indicate to skip BIDS curation for that acquisition.

In [ ]:
for acq in session_object.acquisitions():
    # get the full acquisition object
    acq = fw.get(acq.id)
    acq.update({'label':acq.label+"_ignore-BIDS"})

<br>

Looking for more? Check out our github repository with some commonly used SDK examples! (here)<>

### *Thats all Folks!*